In [1]:
%pip install pandas matplotlib

import pandas as pd
import matplotlib.pyplot as plt


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Erro ao carregar dados: [Errno 2] No such file or directory: '/mnt/data/fonte.csv'


In [56]:
# Função para carregar e exibir informações básicas da base de dados
def carregar_dados(file_path):
    try:
        dados = pd.read_csv(file_path, low_memory=False)
        # print("Dados carregados com sucesso!")
        # print(dados.head())
        return dados
    except Exception as e:
        # print(f"Erro ao carregar dados: {e}")
        return None

# Função para limpar e transformar dados
def limpar_dados(dados):
    ## Exibir info do dataset
    # print(dados.info())
    # Transformar a coluna de data de início de exercício em datetime
    dados['DATA_INICIO_EXERC'] = pd.to_datetime(dados['DATA_INICIO_EXERC'], errors='coerce')
    # # Remover linhas com datas inválidas
    dados = dados.dropna(subset=['DATA_INICIO_EXERC'])
    # Transformar a coluna de ano de nascimento em inteiro
    dados['ANO_NASCIMENTO'] = pd.to_numeric(dados['ANO_NASCIMENTO'], errors='coerce')
    
    return dados

# Função para remover os dados que não serão utilizados
def remover_dados_desnecessarios(dados):

    #
    # Remover colunas desnecessárias

    colunas_excluir = ['VINCULO', 'REF_CARGO_BAS', 'GRUPO', 'REF_CARGO_COM', 'ESCOL_CARGO_COMISSAO', 'JORNADA', 'ORGAO_EXT', 'RACA', 'DEFICIENTE']
    
    dados = dados.drop(columns=colunas_excluir)

    # Remover setores que não são bibliotecas e que não são de interesse  
    setores_excluir = [
    'BIBLIOTECA MUNICIPAL MARIO DE ANDRADE', 
    'BIBLIOTECA PUBLICA MUNICIPAL LOUIS BRAILLE', 
    'BIBLIOTECA PUBLICA MUNICIPAL SERGIO MILLIET',
    'BIBLIOTECA JOAO CABRAL DE MELO NETO - CEU VILA CUR',
    'BIBLIOTECA JORNALISTA ROBERTO MARINHO - CEU BUTANT',
    'BIBLIOTECA RACHEL DE QUEIROZ - CEU ALVARENGA']
    
    dados = dados[dados['SETOR'].str.contains('BIBLIOTECA') & ~dados['SETOR'].isin(setores_excluir)]

    #Remover linhas com setor nulo
    dados = dados.dropna(subset=['SETOR'])
   
    #Exibir info do dataset
    # print(dados.info())
    return dados

def contagem_servidores_por_setor(dados):
    # Contagem de servidores por setor
    contagem_servidores = dados['SETOR'].value_counts()
    contagem_servidores = contagem_servidores.reset_index()
    contagem_servidores.columns = ['SETOR', 'QUANTIDADE_SERVIDORES']
    contagem_servidores = contagem_servidores.sort_values(by='QUANTIDADE_SERVIDORES', ascending=False)
    print(contagem_servidores) 


# Função para realizar a análise
def analisar_dados(dados):
    # Quantidade de servidores que ingressaram antes de uma data específica
    ingressos_antes_reforma = dados[dados['DATA_INICIO_EXERC'] < '2022-03-19']
    print(f"Quantidade de servidores que ingressaram antes de 19/03/2022 (Reforma da previdência): {len(ingressos_antes_reforma)}")
    # Quantidade de servidores que terão mais de 20 anos de serviço até 2024
    
    ##TODO: Verificar se a data de início no df 2024 é o mesmo do df 2019 para todos os servidores, se for diferente considerar a data de início do df 2019
    ingressos_20_anos = ingressos_antes_reforma[ingressos_antes_reforma['DATA_INICIO_EXERC'] < '2004-03-19']
    print(f"Quantidade de servidores que terão mais de 20 anos de serviço até 2024: {len(ingressos_20_anos)}")

    # Análise por sexo e idade
    masc_65 = dados[(dados['SEXO'] == 'M') & (dados['IDADE'] >= 65)]
    fem_62 = dados[(dados['SEXO'] == 'F') & (dados['IDADE'] >= 62)]
    print(f"Quantidade de servidores do sexo masculino que terão 65 anos em 2024: {masc_65.shape[0]}")
    print(f"Quantidade de servidores do sexo feminino que terão 62 anos em 2024: {fem_62.shape[0]}")

# Função para criar visualizações
def criar_visualizacoes(dados):
    # Distribuição por idade
    plt.figure(figsize=(10, 6))
    dados['IDADE'].hist(bins=30)
    plt.title('Distribuição de Idade dos Servidores')
    plt.xlabel('Idade')
    plt.ylabel('Quantidade')
    plt.show()

# Função para estimar servidores em condição de abono até 2029
def estimar_abono_2029(dados):
    # Calcular a idade dos servidores
    dados['IDADE'] = 2024 - dados['ANO_NASCIMENTO']
    # Calcular o tempo de serviço até 2024
    dados['TEMPO_SERVICO_ATE_2024'] = 2024 - dados['DATA_INICIO_EXERC'].dt.year
    # Estimar o tempo de serviço até 2029 (adicionar 5 anos)
    dados['TEMPO_SERVICO_ATE_2029'] = dados['TEMPO_SERVICO_ATE_2024'] + 5

    # Identificar servidores que podem atingir os critérios de abono até 2029
    dados['POTENCIAL_ABONO_2029'] = False
    # Homens com pelo menos 35 anos de serviço
    dados.loc[(dados['SEXO'] == 'M') & (dados['TEMPO_SERVICO_ATE_2029'] >= 35), 'POTENCIAL_ABONO_2029'] = True
    # Mulheres com pelo menos 30 anos de serviço
    dados.loc[(dados['SEXO'] == 'F') & (dados['TEMPO_SERVICO_ATE_2029'] >= 30), 'POTENCIAL_ABONO_2029'] = True

    # Contar servidores que potencialmente estarão em condição de abono até 2029
    servidores_potencial_abono_2029 = dados[dados['POTENCIAL_ABONO_2029']].shape[0]
    print(f"Servidores que podem estar em condição de abono até 2029: {servidores_potencial_abono_2029}")




In [57]:
# Caminho para o arquivo CSV
file_path = r'C:\Repos\tcc-bruna\fonte.csv'

# Executar o fluxo de trabalho
dados = carregar_dados(f'{file_path}')
if dados is not None:
    dados_limpos = limpar_dados(dados)
    bibliotecas = remover_dados_desnecessarios(dados_limpos)
    contagem_servidores_por_setor(bibliotecas)
    # analisar_dados(dados_limpos)
    # criar_visualizacoes(dados_limpos)
    # estimar_abono_2029(dados_limpos)

                                                SETOR  QUANTIDADE_SERVIDORES
0                           SUPERVISAO DE BIBLIOTECAS                     36
1     COORDENACAO DO SISTEMA MUNICIPAL DE BIBLIOTECAS                     14
2        BIBLIOTECA PUBLICA MUNICIPAL MONTEIRO LOBATO                     12
3        BIBLIOTECA PUBLICA MUNICIPAL AFONSO SCHIMIDT                     11
4    BIBLIOTECA PUBLICA MUNICIPAL RAIMUNDO DE MENEZES                      9
5     BIBLIOTECA PUBLICA MUNICIPAL VINICIUS DE MORAES                      8
6          BIBLIOTECA PUBLICA MUNICIPAL RICARDO RAMOS                      8
7   BIBLIOTECA PUBLICA MUNICIPAL PE. JOSE DE ANCHIETA                      8
8   BIBLIOTECA PUBLICA MUNICIPAL JOVINA ROCHA ALVARES                      8
9     BIBLIOTECA PUBLICA MUNICIPAL ALVARES DE AZEVEDO                      8
10  BIBLIOTECA PUBLICA MUNICIPAL HANS CHRISTIAN ANDER                      7
11         BIBLIOTECA PUBLICA MUNICIPAL CORA CORALINA                      7

C:\Users\freda\AppData\Local\Temp\ipykernel_17148\3826319252.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['ANO_NASCIMENTO'] = pd.to_numeric(dados['ANO_NASCIMENTO'], errors='coerce')
